# Regression Analysis

## Does crative features influence on Click Through Rate?

-- Cuáles son los drivers de performance en Paid Social (TikTok)?

In [4]:
import json
import os

# Data manipulation libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Modeling libraries


In [5]:
#load json data and convert to pandas dataframe
def load_json_to_dataframe(json_file):
    with open(json_file, 'r') as file:
        data = json.load(file)
    df = pd.DataFrame(data)
    return df

raw_data = load_json_to_dataframe('/Users/camilojaureguiberry/Documents/Projects/Developments/NarrativeLens/data/datasets/NarrativeLens.tiktok_ads_us_toplikes.json')



In [6]:
raw_data.head()

,_id,ad_title,brand_name,cost,ctr,favorite,id,industry_key,is_search,like,...,tag,video_info,detail_analysis,industry,objective,scrap_datetime,video_name,creative_features,is_ugc,ugc_explanation
0,{'$oid': '67f0473955c66f33e12973f2'},Great time killer!,Survival Game Master,2,0.01,False,7132878852058906625,label_23125000000,True,38213,...,3.0,"{'vid': 'v0911dg40001cbta3ebc77u7vbp6b1gg', 'd...",This ad is using Product Review to catch audie...,{'child': {'id': {'$numberLong': '23125000000'...,"{'id': 2, 'value': 'App Installs'}",2025-04-04T17:53:16.735723,ad_23000000000_2_0_1,{'creative_theme': 'Promotional & Offer-Based'...,True,Justification: This video qualifies as a UGC-s...
1,{'$oid': '67f0473955c66f33e12973f3'},Oddly satisfying game,Gameworld Master,2,0.01,False,7109275920046178305,label_23125000000,True,14299,...,3.0,"{'vid': 'v10033g50000caldhr3c77ub7mthrn5g', 'd...",This ad is using Strategy Focused to catch aud...,{'child': {'id': {'$numberLong': '23125000000'...,"{'id': 2, 'value': 'App Installs'}",2025-04-04T17:53:16.735723,ad_23000000000_2_1_1,"{'creative_theme': 'Product-Centric', 'creativ...",False,Justification: This video does not qualify as ...
2,{'$oid': '67f0473955c66f33e12973f4'},My friend recommended me to play this game,Sugarcane Inc. Empire Tycoon,0,0.02,False,7077499601561305089,label_23125000000,True,2427,...,NaN,"{'vid': 'v10033g50000cfgfgl3c77u9fehpnd8g', 'd...",,{'child': {'id': {'$numberLong': '23125000000'...,"{'id': 2, 'value': 'App Installs'}",2025-04-04T17:53:16.735723,ad_23000000000_2_2_1,"{'creative_theme': 'Educational & Explainer', ...",False,Justification: This video does not meet the UG...
3,{'$oid': '67f0473955c66f33e12973f5'},download now,"Smart VPN - Fast, Stable",2,0.02,False,7176836203668783106,label_23125000000,True,124771,...,3.0,"{'vid': 'v10033g50000cuqnqlnog65qepkj72vg', 'd...",This ad is using Oddly Satisfying to catch aud...,{'child': {'id': {'$numberLong': '23125000000'...,"{'id': 2, 'value': 'App Installs'}",2025-04-04T17:53:16.735723,ad_23000000000_2_3_1,"{'creative_theme': 'Lifestyle & Aspirational',...",False,Justification: This video does not qualify as ...
4,{'$oid': '67f0473955c66f33e12973f6'},Play ten minutes a day to relieve stress!,Sugarcane Factory 3D,0,0.02,False,7077499507373899777,label_23125000000,True,2739,...,NaN,"{'vid': 'v10033g50000cfl0uujc77u563dfii20', 'd...",,{'child': {'id': {'$numberLong': '23125000000'...,"{'id': 2, 'value': 'App Installs'}",2025-04-04T17:53:16.735723,ad_23000000000_2_4_1,"{'creative_theme': 'Product-Centric', 'creativ...",True,Justification: This video demonstrates the gam...
